In [ ]:
# import the necessary packages
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2 #neural network, light enough for mobile devices as well.
from tensorflow.keras.layers import AveragePooling2D # a filter that averages using neighbourhood points of a image.
from tensorflow.keras.layers import Dropout #this layer prevents overfitting of a model.
from tensorflow.keras.layers import Flatten #flattens a tensor/image into one dimension so that to input into dense layers.
from tensorflow.keras.layers import Dense # this is the neural network of the deep learning architecture.
from tensorflow.keras.layers import Input #its the input side of dense layer 
from tensorflow.keras.models import Model #provides the model for training.
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np   ##handles matrix operations in optimized manner.
import os

In [ ]:
from tensorflow.keras.layers import MaxPooling2D

In [ ]:
# initialize the initial learning rate, number of epochs to train for,
# and batch size
INIT_LR = 1e-4
EPOCHS = 75
BS = 32

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
!ls '/content/drive/MyDrive'

In [ ]:
#data set directory.
DIRECTORY = r"drive/MyDrive/dataset"
#categorize the two image sets,
CATEGORIES = ["with_smile", "without_smile"]

# grab the list of images in our dataset directory, then initialize
# the list of data (i.e., images) and class images
print("[INFO] loading images...")

In [ ]:
#empty arrays to store image data and image labels.
data = []
labels = [] # with smile and without smile.

for category in CATEGORIES:
	#accesses with simle image folder and without smile for each category
    path = os.path.join(DIRECTORY, category)
    for img in os.listdir(path):
      img_path = os.path.join(path, img)
      image = load_img(img_path, target_size=(224, 224))
      image = img_to_array(image)
      image = preprocess_input(image)
      data.append(image)
      labels.append(category)

In [ ]:
# perform one-hot encoding on the labels
lb = LabelBinarizer() ##calling labelBinarizer class.
labels = lb.fit_transform(labels) ##converts into binary labels.
labels = to_categorical(labels)   ##for tensorflow to understand.


In [ ]:
##converts into numpy arrays.
data = np.array(data, dtype="float32")
labels = np.array(labels)

In [ ]:
(trainX, testX, trainY, testY) = train_test_split(data, labels,
	test_size=0.20, stratify=labels, random_state=42) ##only 0.2 or 20% is used for testing.

In [ ]:
##so that the model can undertand human face with non ideal camera configurations.
aug = ImageDataGenerator(
	rotation_range=20,
	zoom_range=0.15,
	width_shift_range=0.2,
	height_shift_range=0.2,
	shear_range=0.15,
	horizontal_flip=True,
	fill_mode="nearest")

In [ ]:
# load the MobileNetV2 network, ensuring the head FC layer sets are
# left off
baseModel = MobileNetV2(weights="imagenet", include_top=False,
	input_tensor=Input(shape=(224, 224, 3))) ##224 224 is image size and 3 is color channel., RBG
#baseModel.summary()

In [ ]:
# construct the head of the model that will be placed on top of the
# the base model
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(5, 5))(headModel)
headModel = Flatten(name="flatten")(headModel) ##flattens the 2D array into 1 dimension,
headModel = Dense(256, activation="relu")(headModel) ##an activation function, rectified linera unit.
headModel = Dropout(0.4)(headModel) #Dropout is a regularization method used in artificial neural networks that reduces the risk of overfitting .
#headModel = Dense(128, activation="relu")(headModel)
#headModel = Dropout(0.4)(headModel)
# headModel = Dense(30, activation="relu")(headModel)
headModel = Dense(2, activation="softmax")(headModel)


In [ ]:
maxModel = baseModel.output
maxModel = MaxPooling2D(pool_size=(5,5))(maxModel)
maxModel = Flatten(name="flatten")(maxModel)
maxModel = Dense(256, activation="relu")(maxModel)
maxModel = Dropout(0.4)(maxModel)
maxModel = Dense(2, activation="softmax")(maxModel)

In [ ]:
# loop over all layers in the base model and freeze them so they will
# *not* be updated during the first training process
for layer in baseModel.layers:
	layer.trainable = False

In [ ]:
# place the head FC model on top of the base model (this will become
# the actual model we will train)
model = Model(inputs=baseModel.input, outputs=headModel)
maxModel = Model(inputs=baseModel.input, outputs=maxModel)

In [ ]:
# compile our model
print("[INFO] compiling model...")
#Adam optimization is a stochastic gradient descent method that is based on adaptive estimation of first-order and second-order moments.
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="binary_crossentropy", optimizer=opt,
	metrics=["accuracy"])

maxModel.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])

In [ ]:
	# train the head of the network
print("[INFO] training head Average Pooling Model...")
H = model.fit(
	aug.flow(trainX, trainY, batch_size=BS),
	steps_per_epoch=len(trainX) // BS,
	validation_data=(testX, testY),
	validation_steps=len(testX) // BS,
	epochs=EPOCHS)


In [ ]:
	# train the head of the network
print("[INFO] training head Max Pooling Model...")
H_M = maxModel.fit(
	aug.flow(trainX, trainY, batch_size=BS),
	steps_per_epoch=len(trainX) // BS,
	validation_data=(testX, testY),
	validation_steps=len(testX) // BS,
	epochs=EPOCHS)

In [ ]:
# make predictions on the testing set
print("[INFO] evaluating network...")
predIdxs = model.predict(testX, batch_size=BS)
# for each image in the testing set we need to find the index of the
# label with corresponding largest predicted probability
predIdxs = np.argmax(predIdxs, axis=1)

# show a nicely formatted classification report
print(classification_report(testY.argmax(axis=1), predIdxs,
	target_names=lb.classes_))

# make predictions on the testing set Max Pooling Model
print("[INFO] evaluating Max Pooling network...")
predIdxsMax = maxModel.predict(testX, batch_size=BS)
# for each image in the testing set we need to find the index of the
# label with corresponding largest predicted probability
predIdxsMax = np.argmax(predIdxsMax, axis=1)

# show a nicely formatted classification report
print(classification_report(testY.argmax(axis=1), predIdxsMax,
	target_names=lb.classes_))


# serialize the model to disk
print("[INFO] saving smile detector model...")
maxModel.save("drive/MyDrive/SS_MAX", save_format="h5")
model.save("drive/MyDrive/SS_AVERAGE", save_format="h5")

In [ ]:
# plot the training loss and accuracy
N = EPOCHS
plt.style.use("ggplot")
plt.figure()
plt.figure(figsize=(18, 12))
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
plt.plot(np.arange(0, N), H_M.history["loss"], label="train_loss_max")
plt.plot(np.arange(0, N), H_M.history["val_loss"], label="val_loss_max")
plt.plot(np.arange(0, N), H_M.history["accuracy"], label="train_acc_max")
plt.plot(np.arange(0, N), H_M.history["val_accuracy"], label="val_acc_max")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig("drive/MyDrive/plot.png")

In [ ]:
# plot the training loss and accuracy
N = EPOCHS
plt.style.use("ggplot")
plt.figure()
plt.figure(figsize=(18, 12))
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="Validation accuracy by Average Pooling")
plt.plot(np.arange(0, N), H_M.history["val_accuracy"], label="Validation accuracy by  Max Pooling")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig("drive/MyDrive/val_plot.png")